In [85]:
import numpy as np
import itertools
from functools import total_ordering

In [196]:
lines = [l for l in open("inputs/13.input").read().split("\n") if len(l) >= 1]

In [230]:
# for a curve and current direction return the new direction
curve_mapping = {
    "/": {
        "N": "E",
        "E": "N",
        "S": "W",
        "W": "S"
    },
    "\\": {
        "N": "W",
        "E": "S",
        "S": "E",
        "W": "N"
    }
}

# for an intersection, the current direction and the turn that gets taken return the new direction
intersection_mapping = {
    0: { # turn left
        "N": "W",
        "E": "N",
        "S": "E",
        "W": "S"
    },
    1: { # straight
        "N": "N",
        "E": "E",
        "S": "S",
        "W": "W"
    },
    2: { # turn right
        "N": "E",
        "E": "S",
        "S": "W",
        "W": "N"
    }
}

In [199]:
@total_ordering
class Cart:
    def __init__(self, x, y, dir):
        self.x = x
        self.y = y
        self.dir = dir # N, E, S, W
        self.intersection_counter = 0
        
    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)
    
    def __lt__(self, other):
        # assuming no board is wider than 100000 columns
        return (self.y * 100000 + self.x) < (other.y * 100000 + other.x)
    
    def __str__(self):
        return f"Cart: {self.y} | {self.x}: {self.dir}"
    
    def __repr__(self):
        return str(self)
    
    def copy(self):
        return Cart(self.x, self.y, self.dir)
    
    def move(self, board):
        track = board[(self.y, self.x)]
        if track.type  == "+":
            self.dir = intersection_mapping[self.intersection_counter][self.dir]            
            self.intersection_counter = (self.intersection_counter + 1) % 3
        elif track.type in ["/", "\\"]:
            self.dir = curve_mapping[track.type][self.dir]
        
        if self.dir in ["E", "W"]:
            self.x += (1 if self.dir == "E" else -1)
        else:
            self.y += (1 if self.dir == "S" else -1)

In [200]:
class Track:
    def __init__(self, x, y, type):
        self.x = x
        self.y = y
        self.type = type # |, -, /, \, +
    
    def __str__(self):
        return f"{self.type}:  {self.y} | {self.x}"
    
    def __repr__(self):
        return str(self)

In [210]:
board = {}
initial_carts = []

for y, l in enumerate(lines):
    for x, sym in enumerate(l):
        if sym == " ":
            continue
        elif sym in ["|", "-", "/", "\\", "+"]:
            board[(y, x)] = Track(x, y, sym)
        elif sym == ">":
            board[(y, x)] = Track(x, y, "-")
            initial_carts.append(Cart(x, y, "E"))
        elif sym == "<":
            board[(y, x)] = Track(x, y, "-")
            initial_carts.append(Cart(x, y, "W"))
        elif sym == "^":
            board[(y, x)] = Track(x, y, "|")
            initial_carts.append(Cart(x, y, "N"))
        elif sym == "v":
            board[(y, x)] = Track(x, y, "|")
            initial_carts.append(Cart(x, y, "S"))
        else:
            print("Error: ", sym, y, x)
            assert False

In [211]:
def location_of_crash(carts):
    for c1, c2 in itertools.combinations(carts, 2):
        if c1 == c2:
            return (c1.x, c1.y)

### Part1: First crash printed out here:

In [223]:
board = board.copy()
carts = [c.copy() for c in initial_carts]

while len(carts) > 1:
    #print(carts)
    carts = sorted(carts)
    for cart in carts:
        crash = location_of_crash(carts)
        if crash is not None:
            print("Crash: {},{}".format(*crash))
            carts = [c for c in carts if (c.x, c.y) != crash]
        
        cart.move(board)
    # also check for the last cart
    crash = location_of_crash(carts)
    if crash is not None:
        print("Crash: {},{}".format(*crash))
        carts = [c for c in carts if (c.x, c.y) != crash]

Crash: 41,22
Crash: 112,8
Crash: 73,87
Crash: 52,38
Crash: 109,30
Crash: 39,66
Crash: 98,32
Crash: 56,25


### Part2: Last remaining cart:

In [229]:
print("{},{}".format(carts[0].x, carts[0].y))

84,90
